This notebook generates a PyTorch dataset from a FiftyOne dataset for training

- https://github.com/voxel51/fiftyone-examples/blob/master/examples/pytorch_detection_training.ipynb
- https://towardsdatascience.com/stop-wasting-time-with-pytorch-datasets-17cac2c22fa8
- https://voxel51.com/blog/fiftyone-tips-and-tricks-for-accelerating-computer-vision-workflows-mar-17-2023/
- https://stackoverflow.com/questions/78720028/fiftyone-dataset-with-pytorch-dataloader

In [ ]:
%load_ext autoreload
%autoreload 0

import sys
import os
import os.path
import fiftyone as fo
import importlib

sys.path.append("..")

In [ ]:
import utils.data_loader

importlib.reload(utils.data_loader)
from utils.data_loader import FiftyOneTorchDatasetCOCO, TorchToHFDatasetCOCO

import fiftyone as fo
from fiftyone.utils.huggingface import load_from_hub

try:
    dataset_v51 = load_from_hub("dbogdollumich/mcity_fisheye_v51")
except:
    dataset_v51 = fo.load_dataset("dbogdollumich/mcity_fisheye_v51")

torch_dataset = FiftyOneTorchDatasetCOCO(dataset_v51)
converter_torch_hf = TorchToHFDatasetCOCO(torch_dataset)
hf_dataset = converter_torch_hf.convert()

# Access the Hugging Face dataset directly
print(hf_dataset["train"][0])
print(hf_dataset["val"][0])

In [ ]:
import os

import torch
import fiftyone.utils.coco as fouc
from PIL import Image
import torch
import numpy as np

tokens = {}
with open("/home/dbogdoll/mcity_data_engine/.secret", "r") as file:
    for line in file:
        key, value = line.strip().split("=")
        tokens[key] = value

os.environ["HF_TOKEN"] = tokens["HF_TOKEN"]

In [ ]:
# Load dataset with Huggingface
from datasets import load_dataset

dataset_hf = load_dataset("dbogdollumich/mcity_fisheye_v51", split="train")
dataset_hf[0]["image"]

In [ ]:
# Load dataset with Fiftyone
import fiftyone as fo
from fiftyone.utils.huggingface import load_from_hub

try:
    dataset_v51 = load_from_hub("dbogdollumich/mcity_fisheye_v51")
except:
    dataset_v51 = fo.load_dataset("dbogdollumich/mcity_fisheye_v51")

In [ ]:
class FiftyOneTorchDataset(torch.utils.data.Dataset):
    """A class to construct a PyTorch dataset from a FiftyOne dataset.

    Args:
        fiftyone_dataset: a FiftyOne dataset or view that will be used for training or testing
        transforms (None): a list of PyTorch transforms to apply to images and targets when loading
        gt_field ("ground_truth"): the name of the field in fiftyone_dataset that contains the
            desired labels to load
        classes (None): a list of class strings that are used to define the mapping between
            class names and indices. If None, it will use all classes present in the given fiftyone_dataset.
    """

    def __init__(
        self,
        fiftyone_dataset,
        transforms=None,
        gt_field="ground_truth",
        classes=None,
    ):
        self.samples = fiftyone_dataset
        self.transforms = transforms
        self.gt_field = gt_field

        self.img_paths = self.samples.values("filepath")

        self.classes = classes
        if not self.classes:
            # Get list of distinct labels that exist in the view
            self.classes = self.samples.distinct("%s.detections.label" % gt_field)

        if self.classes[0] != "background":
            self.classes = ["background"] + self.classes

        self.labels_map_rev = {c: i for i, c in enumerate(self.classes)}

    def __getitem__(self, idx):
        img_path = self.img_paths[idx]
        sample = self.samples[img_path]
        metadata = sample.metadata
        img = Image.open(img_path).convert("RGB")

        boxes = []
        labels = []
        area = []
        iscrowd = []
        detections = sample[self.gt_field].detections
        for det in detections:
            category_id = self.labels_map_rev[det.label]
            coco_obj = fouc.COCOObject.from_label(
                det,
                metadata,
                category_id=category_id,
            )
            x, y, w, h = coco_obj.bbox
            boxes.append([x, y, x + w, y + h])
            labels.append(coco_obj.category_id)
            area.append(coco_obj.area)
            iscrowd.append(coco_obj.iscrowd)

        target = {}
        target["boxes"] = torch.as_tensor(boxes, dtype=torch.float32)
        target["labels"] = torch.as_tensor(labels, dtype=torch.int64)
        target["image_id"] = torch.as_tensor([idx])
        target["area"] = torch.as_tensor(area, dtype=torch.float32)
        target["iscrowd"] = torch.as_tensor(iscrowd, dtype=torch.int64)

        if self.transforms is not None:
            img, target = self.transforms(img, target)

        return img, target

    def __getitems__(self, indices):
        samples = [self.__getitem__(idx) for idx in indices]
        return samples

    def __len__(self):
        return len(self.img_paths)

    def get_classes(self):
        return self.classes

    def get_splits(self):
        splits = set()
        for sample in self.samples.iter_samples():
            split = sample.tags[0]  # Assuming the split is the first tag
            splits.add(split)
        return splits

In [ ]:
torch_dataset = FiftyOneTorchDataset(dataset_v51)
print(torch_dataset)
sample = torch_dataset[0]
print(sample)

In [ ]:
# Torch dataset to Huggingface dataset

from datasets import Dataset, NamedSplit


def gen(dataset, split_name):
    for idx in range(len(dataset)):
        img_path = dataset.img_paths[idx]
        sample = dataset.samples[img_path]
        split = sample.tags[0]
        if split != split_name:
            continue
        metadata = sample.metadata

        boxes = []
        labels = []
        area = []
        iscrowd = []
        detections = sample[dataset.gt_field].detections
        for det in detections:
            category_id = dataset.labels_map_rev[det.label]
            coco_obj = fouc.COCOObject.from_label(
                det,
                metadata,
                category_id=category_id,
            )
            x, y, w, h = coco_obj.bbox
            boxes.append([x, y, x + w, y + h])
            labels.append(coco_obj.category_id)
            area.append(coco_obj.area)
            iscrowd.append(coco_obj.iscrowd)

        target = {
            "boxes": boxes,
            "labels": labels,
            "image_id": idx,
            "area": area,
            "iscrowd": iscrowd,
        }

        yield {"image": img_path, "target": target, "split": split}


# Function to create HuggingFace datasets for each split
def create_hf_dataset(torch_dataset):
    splits = torch_dataset.get_splits()
    hf_datasets = {}
    for split in splits:
        hf_datasets[split] = Dataset.from_generator(
            lambda: gen(torch_dataset, split), split=NamedSplit(split)
        )
    return hf_datasets


# Create the HuggingFace datasets
hf_dataset = create_hf_dataset(torch_dataset)

print(hf_dataset["train"][0])
print(hf_dataset["val"][0])